<a href="https://colab.research.google.com/github/Agniswar123/processorlimit/blob/master/ensembled_feature_selection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install skfeature-chappers

  Created wheel for skfeature-chappers: filename=skfeature_chappers-1.0.3-py2.py3-none-any.whl size=59512 sha256=30e28e6c31ffa3899335ed114e7e4861190a357fb81a44eb07d8f302680cc03d
  Stored in directory: /root/.cache/pip/wheels/ac/61/bf/1b3a8c232a0072409508c2ec4c12f316e95681ae72ba7315d2
Successfully built skfeature-chappers


In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import LeaveOneOut
from sklearn.feature_selection import mutual_info_classif
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
#from sklearn.tree import DecisionTreeClassifier   Not used here
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
#read data
path="colon.txt"
data=pd.read_csv(path,delimiter="\t" , index_col=None, header=None)
print(data.head())
print(data.shape)

   0     1     2     3     ...        1997       1998       1999  2000
0  8590  5470  4260  4060  ...   75.699997  83.500000  28.700001     1
1  3830  6970  5370  4710  ...   42.700001  16.100000  15.200000     1
2  3230  3690  3400  3460  ...   57.599998   7.490000  31.799999     1
3  3230  3690  3400  3460  ...   57.599998   7.490000  31.799999     1
4  9330  7020  4720  9490  ...  122.000000  40.400002  26.799999     1

[5 rows x 2001 columns]
(62, 2001)


In [ ]:
#split in input and output
x=data.iloc[:,:-1]
y=data.iloc[:,-1]
samples=(data.shape[0]) 
title=path.split(".")[0]

print(title)

colon


In [ ]:
#gridsearchcv to evaluate parameters of different classifiers
#this nees to be done before apllying filter or any further data split
from sklearn.model_selection import GridSearchCV

#SVM
parameters = {'kernel':('linear', 'poly', 'rbf', 'sigmoid'), 'C':[i for i in range(1,10,1)]}
svc = SVC()

svc_best = GridSearchCV(svc, parameters)
svc_best.fit(X=x, y=y)
print(svc_best.best_estimator_)

#NB dont take any parameters so its not checked
#KNN
parameters = {'weights':('uniform', 'distance'), 'algorithm':('auto','brute','ball_tree','kd_tree'), 'metric':('euclidean','manhattan','chebyshev'), 'n_neighbors':[i for i in range(3,15,2)], 'leaf_size':[j for j in range(30,150,5)], 'p':[k for k in range(1,5,1)]}
knn= KNeighborsClassifier()
knn_best=GridSearchCV(knn,parameters)
knn_best.fit(X=x, y=y)
print(knn_best.best_estimator_)

"""
This function took almost 8-10 mins to execute. thats why the outputs are stored separately and no variable is used
SVC(C=6, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)
KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='manhattan',
                     metric_params=None, n_jobs=None, n_neighbors=3, p=1,
                     weights='uniform')
"""


SVC(C=6, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)
KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='manhattan',
                     metric_params=None, n_jobs=None, n_neighbors=3, p=1,
                     weights='uniform')


In [ ]:
#fisher score function
from scipy.sparse import *
def fisher_score(X, y):
    import numpy as np
    
    from skfeature.utility.construct_W import construct_W
    """
    This function implements the fisher score feature selection, steps are as follows:
    1. Construct the affinity matrix W in fisher score way
    2. For the r-th feature, we define fr = X(:,r), D = diag(W*ones), ones = [1,...,1]', L = D - W
    3. Let fr_hat = fr - (fr'*D*ones)*ones/(ones'*D*ones)
    4. Fisher score for the r-th feature is score = (fr_hat'*D*fr_hat)/(fr_hat'*L*fr_hat)-1

    Input
    -----
    X: {numpy array}, shape (n_samples, n_features)
        input data
    y: {numpy array}, shape (n_samples,)
        input class labels

    Output
    ------
    score: {numpy array}, shape (n_features,)
        fisher score for each feature

    Reference
    ---------
    He, Xiaofei et al. "Laplacian Score for Feature Selection." NIPS 2005.
    Duda, Richard et al. "Pattern classification." John Wiley & Sons, 2012.
    """

    # Construct weight matrix W in a fisherScore way
    kwargs = {"neighbor_mode": "supervised", "fisher_score": True, 'y': y}
    W = construct_W(X, **kwargs)

    # build the diagonal D matrix from affinity matrix W
    D = np.array(W.sum(axis=1))
    L = W
    tmp = np.dot(np.transpose(D), X)
    D = diags(np.transpose(D), [0])
    Xt = np.transpose(X)
    t1 = np.transpose(np.dot(Xt, D.todense()))
    t2 = np.transpose(np.dot(Xt, L.todense()))
    # compute the numerator of Lr
    D_prime = np.sum(np.multiply(t1, X), 0) - np.multiply(tmp, tmp)/D.sum()
    # compute the denominator of Lr
    L_prime = np.sum(np.multiply(t2, X), 0) - np.multiply(tmp, tmp)/D.sum()
    # avoid the denominator of Lr to be 0
    D_prime[D_prime < 1e-12] = 10000
    lap_score = 1 - np.array(np.multiply(L_prime, 1/D_prime))[0, :]

    # compute fisher score from laplacian score, where fisher_score = 1/lap_score - 1
    score = 1.0/lap_score - 1
    return np.transpose(score)

In [ ]:
#signal to noise ratio
#using weighted one-vs-all strategy for multi-class data
def signaltonoise(feature, target, axis = 0, ddof = 0):
  import numpy as np
  classes = np.unique(target)
  if len(feature.shape)<2:
    feature = feature.reshape(-1,1)
  row, _ = feature.shape
  if len(classes) <= 2:
    m = None
    std = 0
    for each in classes:
      idx = np.where(target == each)[0]
      #convinient way of doing m1-m2
      if m is None:
        m = feature.iloc[idx, :].mean(axis)
      else:
        m -= feature.iloc[idx, :].mean(axis)

      #sd1+sd2
      std += feature.iloc[idx, :].std(axis = axis, ddof = ddof)

    return np.asanyarray(m/std)

  else:
    snr_scores = [] #for storing the weighted scores
    #using the one vs all strategy for each class with
    for each in classes:
      idx = np.where(target == each)[0]
      idxn = np.where(target != each)[0]
      m = feature.iloc[idx, :].mean(axis) - feature.iloc[idxn, :].mean(axis)
      std = feature.iloc[idx, :].std(axis = axis, ddof = ddof) + feature.iloc[idxn, :].std(axis = axis, ddof = ddof) 
      snr_scores.append((m/std) * len(idx)/row) #weighted snr

    return np.asanyarray(snr_scores).sum(axis = axis)

In [ ]:
#this function is needed during polling, 
#if else cannot handle dataset more than 2 class labels
def mostFrequent(arr): 
  
    # Sort the array 
    arr.sort() 
    n=len(arr)
    # find the max frequency using linear traversal 
    max_count = 1; res = arr[0]; curr_count = 1
      
    for i in range(1, n):  
        if (arr[i] == arr[i - 1]): 
            curr_count += 1
              
        else : 
            if (curr_count > max_count):  
                max_count = curr_count 
                res = arr[i - 1] 
              
            curr_count = 1
      
    # If last element is most frequent 
    if (curr_count > max_count): 
      
        max_count = curr_count 
        res = arr[n - 1] 
      
    return int(res) 

In [ ]:
def feature_ranking(score):
    """
    Rank features in descending order according to fisher score, the larger the fisher score, the more important the
    feature is
    """
    idx = np.argsort(score, 0)
    return idx[::-1]

In [ ]:
#finding Mutual Information score of data.
MI=mutual_info_classif(x,y)

#finding Signal to Noise ratio score of data.
SNR=signaltonoise(x,y)

#finding Fisher Score score of data.
FS=fisher_score(x.values,y)

#selecting best n features.
n=5

#after sorting the score array, first n gene indices are taken
MI_indices=feature_ranking(MI)[:n]
SNR_indices=feature_ranking(SNR)[:n]
FS_indices=feature_ranking(FS)[:n]

#result
print(MI_indices,"\n",SNR_indices,"\n",FS_indices)

[ 624 1866 1670  512 1934] 
 [ 248  764  492 1422  244] 
 [ 248  764  492 1422  244]


In [ ]:
#create subset feature of 3 scoring functions
#y remains same for all, only x changes

x_mi=[]
x_snr=[]
x_fs=[]

#appending the columns to a list
for i in range(n):  #from previous cell, n is no of selected genes
  x_mi.append(x.iloc[:,MI_indices[i]])
  x_snr.append(x.iloc[:,SNR_indices[i]])
  x_fs.append(x.iloc[:,FS_indices[i]])

#converting list to dataframe and then transposing it as the data was added rowwise bt we need columnwise
x_mi=pd.DataFrame(x_mi)
x_mi=x_mi.transpose()
x_snr=pd.DataFrame(x_snr)
x_snr=x_snr.transpose()
x_fs=pd.DataFrame(x_fs)
x_fs=x_fs.transpose()

#print(x_mi,x_snr,x_fs)

In [ ]:
#objects of used classifiers
var=int(samples**0.5)
if var%2==0:
  var=var+1
#use odd no of neighbours for voting and taking sqrt(n) neighbours
KNN_mi = KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='manhattan',
                     metric_params=None, n_jobs=None, n_neighbors=3, p=1,
                     weights='uniform')
KNN_snr = KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='manhattan',
                     metric_params=None, n_jobs=None, n_neighbors=3, p=1,
                     weights='uniform')
KNN_fs = KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='manhattan',
                     metric_params=None, n_jobs=None, n_neighbors=3, p=1,
                     weights='uniform') 
SVM_mi = SVC(C=6, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)
SVM_snr = SVC(C=6, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)
SVM_fs = SVC(C=6, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)
NB_mi = GaussianNB()
NB_snr = GaussianNB()
NB_fs = GaussianNB()
#Tree = DecisionTreeClassifier()
no_filter,no_classifier=3,3
#the ensembled classifier based on results of these classifiers, so these results are all we need

In [ ]:
loo=LeaveOneOut()

ensemble=[]
score=0
#it can be further optimized as y is not modified
fname=title+str(n)+"_genes.txt"
f=open(fname,"w")
f.write("KNN\tSVM\tNB\tEnsemble\tActual\n")

for train_index, test_index in loo.split(x_mi):
  
  x_mi_train_data,y_mi_train=x_mi.iloc[train_index,:],y[train_index]
  x_mi_test_data,y_mi_test=x_mi.iloc[test_index,:],y[test_index]
  KNN_mi.fit(x_mi_train_data,y_mi_train)
  SVM_mi.fit(x_mi_train_data,y_mi_train)
  NB_mi.fit(x_mi_train_data,y_mi_train)
  
  x_snr_train_data,y_snr_train=x_snr.iloc[train_index,:],y[train_index]
  x_snr_test_data,y_snr_test=x_snr.iloc[test_index,:],y[test_index]
  KNN_snr.fit(x_snr_train_data,y_snr_train)
  SVM_snr.fit(x_snr_train_data,y_snr_train)
  NB_snr.fit(x_snr_train_data,y_snr_train)

  x_fs_train_data,y_fs_train=x_fs.iloc[train_index,:],y[train_index]
  x_fs_test_data,y_fs_test=x_fs.iloc[test_index,:],y[test_index]
  KNN_fs.fit(x_fs_train_data,y_fs_train)
  SVM_fs.fit(x_fs_train_data,y_fs_train)
  NB_fs.fit(x_fs_train_data,y_fs_train)

  
  classifier_mi=[KNN_mi,SVM_mi,NB_mi]
  ind_res_mi=[]

  classifier_snr=[KNN_snr,SVM_snr,NB_snr]
  ind_res_snr=[]

  classifier_fs=[KNN_fs,SVM_fs,NB_fs]
  ind_res_fs=[]

  ind_res=np.zeros((no_filter,no_classifier))   #3 filter, 3 classifiers
  #here test data is tested against 3 classifiers, no cross set test data has been used
  for i in range(len(classifier_mi)):
    tmp=[]
    y_mi_predict=classifier_mi[i].predict(x_mi_test_data)
    y_snr_predict=classifier_snr[i].predict(x_snr_test_data)
    y_fs_predict=classifier_fs[i].predict(x_fs_test_data)
    #data getting stored as KNN: midata, snrdata, fsdata
    ind_res[i][0]=y_mi_predict
    ind_res[i][1]=y_snr_predict
    ind_res[i][2]=y_fs_predict

  # checking KNN: MI,SNR,FS and taking combined result over 3 types of sub-dataset
  classifier_indiv=[]
  #this counting is the ensemble part. the more classifiers support a decision, that one is taken 
  for i in range(no_classifier): #this is no of classifiers
    tmp=ind_res[i].tolist()
    var=mostFrequent(tmp)
    classifier_indiv.append(var)
    f.write("%d\t\t"%var)

  # taking result of 3 classifiers and taking their combined result
  var=mostFrequent(classifier_indiv)
  ensemble.append(var)
  f.write("%d\t\t%d\n"%(var,y_mi_test.iloc[0]))
  if var==y_mi_test.iloc[0]:
    score+=1

  #write in a file

 
f.write("\n\nScore: %d\tTotal: %d\nAccuracy: %f"%(score,samples,score/samples))
f.close()
print(ensemble,"\nScore: ",score,"\nTotal: ",samples,"\nAccuracy: ",score/samples)

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0] 
Score:  52 
Total:  62 
Accuracy:  0.8387096774193549
